In [73]:
import numpy as np
import pandas as pd
from plotnine import ggplot
import matplotlib.pyplot as plt
%config Completer.use_jedi = False

In [2]:
data_df = pd.read_csv('C://Users//wrigh//Code//stratascratch//city_supply_demand//datasets/dataset_1.csv')

In [3]:
data_df

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
0,10-Sep-12,7,5,0,2,2,9
1,NaN,8,6,0,2,2,14
2,NaN,9,8,3,0,0,14
3,NaN,10,9,2,0,1,14
4,NaN,11,11,1,4,4,11
...,...,...,...,...,...,...,...
331,NaN,2,3,3,0,2,0
332,NaN,3,3,3,0,1,0
333,NaN,4,1,1,0,0,0
334,NaN,5,4,2,1,1,3


In [4]:
# forward fill Date column
data_df = data_df.fillna(method='ffill')

In [5]:
data_df.tail(20)

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
316,23-Sep-12,11,20,3,3,5,8
317,23-Sep-12,12,25,3,2,4,8
318,23-Sep-12,13,15,4,3,3,9
319,23-Sep-12,14,20,2,4,4,7
320,23-Sep-12,15,22,4,0,2,9
321,23-Sep-12,16,22,0,7,8,11
322,23-Sep-12,17,31,9,4,6,12
323,23-Sep-12,18,36,4,8,11,12
324,23-Sep-12,19,32,9,4,6,11
325,23-Sep-12,20,29,6,3,3,7


## Question 1
**Which date had the most completed trips during the two week period?**

In [6]:
# agg dates by sum completed trips
df_agg_date = data_df.groupby('Date').sum().reset_index()
df_agg_date.nlargest(1, 'Completed Trips ')

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
12,22-Sep-12,276,960,219,248,344,292


## Question 2

**What was the highest number of completed trips within a 24 hour period?**

In [7]:
# we need to resample but this function needs timestamp index
def create_timestamp(date, time):
    return pd.to_datetime(f"{date} {time}:00")

data_df['Timestamp'] = data_df.apply(lambda row: create_timestamp(row['Date'], row['Time (Local)']), axis=1)

In [8]:
data_df

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Timestamp
0,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00
1,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00
2,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00
3,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00
4,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00
...,...,...,...,...,...,...,...,...
331,24-Sep-12,2,3,3,0,2,0,2012-09-24 02:00:00
332,24-Sep-12,3,3,3,0,1,0,2012-09-24 03:00:00
333,24-Sep-12,4,1,1,0,0,0,2012-09-24 04:00:00
334,24-Sep-12,5,4,2,1,1,3,2012-09-24 05:00:00


In [9]:
# calculate rolling sums with 24 hour period
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=24)
df_agg_twentyfour_hrs = data_df.set_index("Timestamp").rolling(window=indexer, min_periods=1).sum()
df_agg_twentyfour_hrs.nlargest(1, 'Completed Trips ')

C:\Users\wrigh\AppData\Local\Temp\ipykernel_15176\3286238798.py:3: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Date'], dtype='object')


,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
Timestamp,,,,,,
2012-09-21 17:00:00,276.0,1015.0,207.0,278.0,367.0,349.0


In [10]:
# parse time interval to output
time_format = "%Y/%m/%d %H:%M"
df_agg_twentyfour_hrs['Time Interval']  = df_agg_twentyfour_hrs.index.strftime(time_format) +  "-" + (df_agg_twentyfour_hrs.index + pd.Timedelta('24 hours')).strftime(time_format)

In [11]:
df_agg_twentyfour_hrs

,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Time Interval
Timestamp,,,,,,,
2012-09-10 07:00:00,276.0,212.0,38.0,29.0,39.0,150.0,2012/09/10 07:00-2012/09/11 07:00
2012-09-10 08:00:00,276.0,217.0,38.0,29.0,39.0,146.0,2012/09/10 08:00-2012/09/11 08:00
2012-09-10 09:00:00,276.0,222.0,40.0,27.0,37.0,138.0,2012/09/10 09:00-2012/09/11 09:00
2012-09-10 10:00:00,276.0,229.0,39.0,27.0,37.0,130.0,2012/09/10 10:00-2012/09/11 10:00
2012-09-10 11:00:00,276.0,232.0,38.0,28.0,37.0,124.0,2012/09/10 11:00-2012/09/11 11:00
...,...,...,...,...,...,...,...
2012-09-24 02:00:00,20.0,20.0,10.0,3.0,6.0,10.0,2012/09/24 02:00-2012/09/25 02:00
2012-09-24 03:00:00,18.0,17.0,7.0,3.0,4.0,10.0,2012/09/24 03:00-2012/09/25 03:00
2012-09-24 04:00:00,15.0,14.0,4.0,3.0,3.0,10.0,2012/09/24 04:00-2012/09/25 04:00


In [12]:
completed_trips = df_agg_twentyfour_hrs.nlargest(1, 'Completed Trips ')['Completed Trips '].values[0]
time_interval = df_agg_twentyfour_hrs.nlargest(1, 'Completed Trips ')['Time Interval'].values[0]

In [13]:
completed_trips

278.0

In [14]:
time_interval

'2012/09/21 17:00-2012/09/22 17:00'

## Question 3 

**Which hour of the day had the most requests during the two week period?**

In [15]:
df_agg_time = data_df.groupby('Time (Local)').sum().reset_index()
df_agg_time.nlargest(1, 'Requests ')

C:\Users\wrigh\AppData\Local\Temp\ipykernel_15176\2879656998.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.


,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
23,23,488,193,126,184,119


## Question 4
**What percentages of all zeroes during the two week period occurred on weekend (Friday at 5 pm to Sunday at 3 am)?**

In [16]:
total_zeroes = data_df['Zeroes '].sum()
total_zeroes

1429

In [17]:
# all zeroes in weekend
data_df['Day'] = pd.to_datetime(data_df['Date']).dt.dayofweek
weekend_zeroes = data_df[((data_df['Day'] == 4) & (data_df['Time (Local)'] >= 17)) 
                          | (data_df['Day'] == 5) |
                          ((data_df['Day'] == 6) & (data_df['Time (Local)'] < 3))]['Zeroes '].sum()
weekend_zeroes

641

In [18]:
# calculate percentage 
weekend_pct = weekend_zeroes / total_zeroes * 100
print(weekend_pct, '%')

44.856543037088876 %


# Question 5

**What is the weighted average ratio of completed trips per driver during the two week period?**

In [19]:
# calculate ratio
data_df['completed_trip_ratio_per_driver'] = data_df['Completed Trips '] / data_df['Unique Drivers']
data_df

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Timestamp,Day,completed_trip_ratio_per_driver
0,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00,0,0.222222
1,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00,0,0.142857
2,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00,0,0.000000
3,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00,0,0.000000
4,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00,0,0.363636
...,...,...,...,...,...,...,...,...,...,...
331,24-Sep-12,2,3,3,0,2,0,2012-09-24 02:00:00,0,NaN
332,24-Sep-12,3,3,3,0,1,0,2012-09-24 03:00:00,0,NaN
333,24-Sep-12,4,1,1,0,0,0,2012-09-24 04:00:00,0,NaN
334,24-Sep-12,5,4,2,1,1,3,2012-09-24 05:00:00,0,0.333333


In [20]:
# drop rows with nan
dataset_without_zero_unique_drivers = data_df[data_df['Unique Drivers'] > 0]
# calc weighted average by giving weight as day's completed trip ratio * all completed trips
weighted_average_ratio = np.average(dataset_without_zero_unique_drivers['completed_trip_ratio_per_driver'] * (dataset_without_zero_unique_drivers['Completed Trips '] / dataset_without_zero_unique_drivers['Completed Trips '].sum())) 
weighted_average_ratio

0.002788813372084982

# Question 6

**In drafting a driver schedule in terms of 8 hours shifts, when are the busiest 8 consecutive hours over the two week period in terms of unique requests? A new shift starts in every 8 hours. Assume that a driver will work same shift each day.**

In [21]:
data_df

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Timestamp,Day,completed_trip_ratio_per_driver
0,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00,0,0.222222
1,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00,0,0.142857
2,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00,0,0.000000
3,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00,0,0.000000
4,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00,0,0.363636
...,...,...,...,...,...,...,...,...,...,...
331,24-Sep-12,2,3,3,0,2,0,2012-09-24 02:00:00,0,NaN
332,24-Sep-12,3,3,3,0,1,0,2012-09-24 03:00:00,0,NaN
333,24-Sep-12,4,1,1,0,0,0,2012-09-24 04:00:00,0,NaN
334,24-Sep-12,5,4,2,1,1,3,2012-09-24 05:00:00,0,0.333333


In [22]:
df_agg_eight_hrs = data_df.set_index(['Timestamp']).resample('8H').sum()
df_agg_eight_hrs.head()

C:\Users\wrigh\AppData\Local\Temp\ipykernel_15176\639547484.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.


,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Day,completed_trip_ratio_per_driver
Timestamp,,,,,,,,
2012-09-10 00:00:00,7,5,0,2,2,9,0,0.222222
2012-09-10 08:00:00,92,78,10,9,11,89,0,0.831169
2012-09-10 16:00:00,156,106,16,15,21,40,0,3.047619
2012-09-11 00:00:00,28,33,12,5,7,17,8,1.400000
2012-09-11 08:00:00,92,108,11,5,6,58,8,0.690079


In [23]:
time_format = "%Y/%m/%d %H:%M"
df_agg_eight_hrs['Time Interval']  = df_agg_eight_hrs.index.strftime(time_format) +  "-" + (df_agg_eight_hrs.index + pd.Timedelta('8 hours')).strftime(time_format)
df_agg_eight_hrs.head()

,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Day,completed_trip_ratio_per_driver,Time Interval
Timestamp,,,,,,,,,
2012-09-10 00:00:00,7,5,0,2,2,9,0,0.222222,2012/09/10 00:00-2012/09/10 08:00
2012-09-10 08:00:00,92,78,10,9,11,89,0,0.831169,2012/09/10 08:00-2012/09/10 16:00
2012-09-10 16:00:00,156,106,16,15,21,40,0,3.047619,2012/09/10 16:00-2012/09/11 00:00
2012-09-11 00:00:00,28,33,12,5,7,17,8,1.400000,2012/09/11 00:00-2012/09/11 08:00
2012-09-11 08:00:00,92,108,11,5,6,58,8,0.690079,2012/09/11 08:00-2012/09/11 16:00


In [24]:
df_agg_eight_hrs.nlargest(1, 'Requests ')

,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Day,completed_trip_ratio_per_driver,Time Interval
Timestamp,,,,,,,,,
2012-09-21 16:00:00,156,515,88,153,192,198,32,6.357265,2012/09/21 16:00-2012/09/22 00:00


In [25]:
df_agg_eight_hrs.nlargest(1, 'Requests ')['Time Interval']

Timestamp
2012-09-21 16:00:00    2012/09/21 16:00-2012/09/22 00:00
Freq: 8H, Name: Time Interval, dtype: object

## Question 7

**True or False: Driver supply always increases when demand increases during the two week period**

In [32]:
# need to plot this to see
difference_df = pd.DataFrame(data_df['Timestamp'])
# calculate request differences to see where it increases
difference_df['request_diff'] = data_df['Requests '].shift(-1) - data_df['Requests ']
# same for drivers
difference_df['driver_diff'] = data_df['Unique Drivers'].shift(-1) - data_df['Unique Drivers']
difference_df.head()

,Timestamp,request_diff,driver_diff
0,2012-09-10 07:00:00,0.0,5.0
1,2012-09-10 08:00:00,-2.0,0.0
2,2012-09-10 09:00:00,1.0,0.0
3,2012-09-10 10:00:00,3.0,-3.0
4,2012-09-10 11:00:00,-2.0,0.0


In [33]:
# no, look at row 3

## Question 8 

**In which 72 hour period is the ratio of Zeroes to Eyeballs the highest?**

In [37]:
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=72)
df_agg_72_hrs = data_df.set_index("Timestamp").rolling(window=indexer, min_periods=1).sum()
df_agg_72_hrs['zeros_to_eyeballs_ratio'] = df_agg_72_hrs['Zeroes '] / df_agg_72_hrs['Eyeballs ']

C:\Users\wrigh\AppData\Local\Temp\ipykernel_15176\1758854818.py:2: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Date'], dtype='object')


In [41]:
time_format = "%Y/%m/%d %H:%M"
df_agg_72_hrs['Time Interval'] = df_agg_72_hrs.index.strftime(time_format) + "-" + (df_agg_72_hrs.index + pd.Timedelta('3 days')).strftime(time_format)
df_agg_72_hrs

,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Day,completed_trip_ratio_per_driver,zeros_to_eyeballs_ratio,Time Interval
Timestamp,,,,,,,,,,
2012-09-10 07:00:00,828.0,956.0,172.0,159.0,211.0,473.0,93.0,20.807776,0.179916,2012/09/10 07:00-2012/09/13 07:00
2012-09-10 08:00:00,828.0,961.0,173.0,158.0,210.0,468.0,96.0,20.835553,0.180021,2012/09/10 08:00-2012/09/13 08:00
2012-09-10 09:00:00,828.0,964.0,174.0,157.0,209.0,462.0,99.0,20.817696,0.180498,2012/09/10 09:00-2012/09/13 09:00
2012-09-10 10:00:00,828.0,970.0,175.0,157.0,209.0,457.0,102.0,20.817696,0.180412,2012/09/10 10:00-2012/09/13 10:00
2012-09-10 11:00:00,828.0,976.0,176.0,159.0,210.0,453.0,105.0,21.017696,0.180328,2012/09/10 11:00-2012/09/13 11:00
...,...,...,...,...,...,...,...,...,...,...
2012-09-24 02:00:00,20.0,20.0,10.0,3.0,6.0,10.0,0.0,0.619048,0.500000,2012/09/24 02:00-2012/09/27 02:00
2012-09-24 03:00:00,18.0,17.0,7.0,3.0,4.0,10.0,0.0,0.619048,0.411765,2012/09/24 03:00-2012/09/27 03:00
2012-09-24 04:00:00,15.0,14.0,4.0,3.0,3.0,10.0,0.0,0.619048,0.285714,2012/09/24 04:00-2012/09/27 04:00


In [42]:
df_agg_72_hrs.nlargest(1, 'zeros_to_eyeballs_ratio')

,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Day,completed_trip_ratio_per_driver,zeros_to_eyeballs_ratio,Time Interval
Timestamp,,,,,,,,,,
2012-09-24 01:00:00,21.0,27.0,17.0,3.0,6.0,10.0,0.0,0.619048,0.62963,2012/09/24 01:00-2012/09/27 01:00


## Question 9

**If you could add 5 drivers to any single hour of every day during the two week period, which hour should you add them to?**

In [45]:
df_agg_time = data_df.groupby('Time (Local)').sum().reset_index()
df_agg_time['eyeball_to_driver_ratio'] = df_agg_time['Eyeballs '] / df_agg_time['Unique Drivers']
df_agg_time

C:\Users\wrigh\AppData\Local\Temp\ipykernel_15176\983304148.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.


,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Day,completed_trip_ratio_per_driver,eyeball_to_driver_ratio
0,0,339,81,102,142,111,42,8.519786,3.054054
1,1,207,43,71,96,94,42,7.555952,2.202128
2,2,170,52,61,100,62,42,7.588889,2.741935
3,3,83,30,21,35,40,42,2.133929,2.075000
4,4,29,21,2,9,9,42,2.000000,3.222222
5,5,42,31,4,14,11,42,NaN,3.818182
6,6,78,32,19,28,37,42,6.152381,2.108108
7,7,111,30,19,22,60,42,5.157937,1.850000
8,8,151,30,24,29,95,42,4.293543,1.589474
9,9,202,39,20,26,110,42,3.340476,1.836364


In [46]:
# and now we filter to where that ratio is greatest (highest unmet demand)
df_agg_time.nlargest(1, 'eyeball_to_driver_ratio')

,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Day,completed_trip_ratio_per_driver,eyeball_to_driver_ratio
23,23,488,193,126,184,119,42,11.269306,4.10084


## Question 10

**True or False: There is exactly two weeks of data in this analysis**

In [60]:
data_df['Timestamp'][0]

Timestamp('2012-09-10 07:00:00')

In [64]:
(data_df['Timestamp'][data_df.shape[0]-1] - data_df['Timestamp'][0])

Timedelta('13 days 23:00:00')

In [50]:
data_df.head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Timestamp,Day,completed_trip_ratio_per_driver
0,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00,0,0.222222
1,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00,0,0.142857
2,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00,0,0.000000
3,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00,0,0.000000
4,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00,0,0.363636


## Question 11

**Looking at the data from all two weeks, which time might make the most sense to consider a true "end day" instead of midnight? (i.e when are supply and demand at both their natural minimums)**

In [65]:
df_min_supply_per_day = data_df.groupby(['Date'])['Unique Drivers'].min().reset_index().rename(columns={"Unique Drivers": "min_supply_per_day"})
df_min_demand_per_day = data_df.groupby(['Date'])['Requests '].min().reset_index().rename(columns={"Requests ": "min_demand_per_day"})

# merge these based on date
supply_demand = pd.merge(df_min_supply_per_day, df_min_demand_per_day, on='Date')
supply_demand

,Date,min_supply_per_day,min_demand_per_day
0,10-Sep-12,3,0
1,11-Sep-12,0,0
2,12-Sep-12,0,0
3,13-Sep-12,0,0
4,14-Sep-12,0,1
5,15-Sep-12,0,0
6,16-Sep-12,0,0
7,17-Sep-12,0,0
8,18-Sep-12,0,0
9,19-Sep-12,0,0


In [67]:
# mrege with initial dataset so we can get other columns
dataset_w_min_supply_demand = pd.merge(data_df, supply_demand, on='Date')
dataset_w_min_supply_demand

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Timestamp,Day,completed_trip_ratio_per_driver,min_supply_per_day,min_demand_per_day
0,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00,0,0.222222,3,0
1,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00,0,0.142857,3,0
2,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00,0,0.000000,3,0
3,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00,0,0.000000,3,0
4,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00,0,0.363636,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
331,24-Sep-12,2,3,3,0,2,0,2012-09-24 02:00:00,0,NaN,0,0
332,24-Sep-12,3,3,3,0,1,0,2012-09-24 03:00:00,0,NaN,0,0
333,24-Sep-12,4,1,1,0,0,0,2012-09-24 04:00:00,0,NaN,0,0
334,24-Sep-12,5,4,2,1,1,3,2012-09-24 05:00:00,0,0.333333,0,0


In [68]:
# check the exact matches with natural minimums
min_point = dataset_w_min_supply_demand[(dataset_w_min_supply_demand.min_supply_per_day == dataset_w_min_supply_demand['Unique Drivers']) & 
                           (dataset_w_min_supply_demand.min_demand_per_day == dataset_w_min_supply_demand['Requests '])]['Time (Local)'].mode()
min_point

0    4
Name: Time (Local), dtype: int64

In [72]:
# so min point is 4 am
# plot and annotate this hour
df_agg_time = data_df.groupby('Time (Local)').sum().reset_index()
min_point_value = df_agg_time['Requests '][min_point[0]]
plt.figuure(figsize=(12,8))

C:\Users\wrigh\AppData\Local\Temp\ipykernel_15176\1671515084.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.


NameError: name 'plt' is not defined